In [1]:
import numpy as np
import pandas as pd
from google.cloud import storage
import time

### Read and union all telematics data

In [2]:
start = time.time()
df1 = pd.read_csv('gs://grab_safety/safety/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df2 = pd.read_csv('gs://grab_safety/safety/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df3 = pd.read_csv('gs://grab_safety/safety/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df4 = pd.read_csv('gs://grab_safety/safety/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df5 = pd.read_csv('gs://grab_safety/safety/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df6 = pd.read_csv('gs://grab_safety/safety/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df7 = pd.read_csv('gs://grab_safety/safety/features/part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df8 = pd.read_csv('gs://grab_safety/safety/features/part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df9 = pd.read_csv('gs://grab_safety/safety/features/part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df10 = pd.read_csv('gs://grab_safety/safety/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
end = time.time()

In [3]:
print("Reading Time:", end - start, "seconds")

Reading Time: 144.47653484344482 seconds


In [4]:
# union all available telematics data
tele = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index=True)

In [5]:
# sort them by bookingID
tele = tele.sort_values(by=["bookingID"]).reset_index(drop=True)

In [22]:
tele.shape

(16135561, 11)

In [24]:
len(tele.bookingID.unique())

20000

We have 20,000 trips with 15135561 telematics data.

### Identify & remove conflicting trips (Being labeled 0 & 1 at the same time)

In [6]:
label = pd.read_csv('gs://grab_safety/safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv')

In [7]:
# identify all bookingID with conflicting labels (multiple labels)
conflict_trips = label.loc[label.bookingID.duplicated(), 'bookingID']

In [18]:
print("Number of conflicting trips:", len(conflict_trips))
print("Number of telematics data involved:", tele.loc[tele.bookingID.isin(conflict_trips), 'bookingID'].value_counts().sum())

Number of conflicting trips: 18
Number of telematics data involved: 18857


There are 18 trips with conflicting labels. 

### Data Cleaning (Removed observations associated with conflicted trips)

In [19]:
# feature data
tele_new = tele[~tele['bookingID'].isin(conflict_trips)]
label_new = label[~label['bookingID'].isin(conflict_trips)]

In [20]:
tele_new.shape

(16116704, 11)

In [26]:
label_new.shape

(19982, 2)

We are now left with 19982 trips with 16116704 telematics data.

### Sub-sample 10% of the trips for benchmarking purpose

We subsample the data of 10% of the trips as a hold-out dataset. We'll use this for testing purpose.

In [12]:
# 24.9374% of the trips are labelled dangerous
label_new['label'].describe()

count    19982.000000
mean         0.249374
std          0.432662
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: label, dtype: float64

The training data contains around 25% of trips tagged as 'dangerous'. Thus, an imbalanced data.

In [27]:
# sample 10% of the trips such that the proportion of dangerous trips is same as population (24.9374%)
sample_trips= label_new.groupby('label').apply(lambda x: x.sample(frac=0.1, random_state=88))
sample_trips.head()

bookingID  label
label                            
0     3494   1030792151056      0
      12143   249108103188      0
      5987   1666447310919      0
      13646    34359738484      0
      4828   1133871366198      0

In [32]:
# sub-sample features data 
test_tele = tele_new[tele_new.bookingID.isin(sample_trips.bookingID)]
train_tele = tele_new[~tele_new.bookingID.isin(test_tele.bookingID)]

In [35]:
# output train and test (hold-out) datasets
test_tele.to_csv('benchmark/bm_tele.csv', index=False)
test_tele.to_csv('data/ori_tele.csv', index=False)

From here on, all exploratory, feature engineering and train-validate process will be done on ori_tele.csv only